In [55]:
import bokeh.io
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from bokeh.io import show
from bokeh.plotting import Figure
from bokeh.models import Plot, Range1d, MultiLine, Line, Circle, HoverTool, BoxZoomTool, ResetTool, PanTool, WheelZoomTool, ColumnDataSource, Legend, LegendItem
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4, Set3, Set3_12

bokeh.io.output_notebook()

Loading BokehJS ...

In [10]:
df = pd.read_csv("data/fullevents.csv")

players = np.unique(df["DestinationPlayerID"][~pd.isnull(df.DestinationPlayerID)])

events = ['Free Kick', 'Duel', 'Pass', 'Others on the ball', 'Foul', 'Goalkeeper leaving line', 'Offside', 'Save attempt', 'Shot', 'Substitution', 'Interruption']
event_colors = {events[i] : Set3_12[i] for i in range(len(events))}

0        #8dd3c7
1        #ffffb3
2        #ffffb3
3        #8dd3c7
4        #bebada
          ...   
59266    #fb8072
59267    #8dd3c7
59268    #ffffb3
59269    #ffffb3
59270    #d9d9d9
Name: EventType, Length: 59271, dtype: object

In [47]:
g = nx.Graph()
#g.add_nodes_from(players)

for i in players:
    #colors of nodes can be changed here
    if 'Husk' in i:
        c = Spectral4[1]
    else: c = Spectral4[0]
    
    g.add_node(i, color=c)

In [48]:
color_list = df.EventType.map(event_colors)

for index, row in df.iterrows():
    if not pd.isnull(row.DestinationPlayerID):
        g.add_edge(row.OriginPlayerID, row.DestinationPlayerID, event=row.EventType, edge_color=color_list[index])

In [68]:
plot = Figure(plot_width=800, plot_height=600,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
#plot.title.text = "Graph Interaction Demonstration"
plot.xgrid.visible = False
plot.ygrid.visible = False

node_hover_tool = HoverTool(tooltips=[("index", "@index")])
plot.add_tools(node_hover_tool)
#plot.add_tools(node_hover_tool, PanTool(), WheelZoomTool(), ResetTool())

graph_renderer = from_networkx(g, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="color")

graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width=2)
plot.renderers.append(graph_renderer)

for n, c in event_colors.items(): #this is retarded
    plot.line(0, 0, color=c, legend_label=n)

#legend = Legend(items=[LegendItem(label=name, renderers=[Line(line_color=color, line_alpha=0.8, line_width=2)]) for (name, color) in event_colors.items()])

show(plot)

In [89]:
gDuel = nx.Graph()

for i in players:
    #colors of nodes can be changed here
    if 'Husk' in i:
        c = Spectral4[1]
    else:
        c = Spectral4[0]
        break
    
    gDuel.add_node(i, color=c)

In [90]:
for index, row in df.iterrows():
    if (row.EventType == "Duel" and df.EventType[index+1] == "Duel"):
        dueler = df.OriginPlayerID[index + 1]
        if ('Husk' in row.OriginPlayerID) and ('Husk' in dueler):
            gDuel.add_edge(row.OriginPlayerID, dueler, edge_color=Spectral4[2])

In [108]:
plotDuel = Figure(plot_width=800, plot_height=600,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
#plot.title.text = "Graph Interaction Demonstration"
plotDuel.xgrid.visible = False
plotDuel.ygrid.visible = False

node_hover_tool = HoverTool(tooltips=[("index", "@index")])
plotDuel.add_tools(node_hover_tool)
#plot.add_tools(node_hover_tool, PanTool(), WheelZoomTool(), ResetTool())

graph_renderer = from_networkx(gDuel, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="color")

graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width=1)
plotDuel.renderers.append(graph_renderer)

#for n, c in event_colors.items(): #this is retarded
#    plotDuel.line(0, 0, color=c, legend_label=n)

#legend = Legend(items=[LegendItem(label=name, renderers=[Line(line_color=color, line_alpha=0.8, line_width=2)]) for (name, color) in event_colors.items()])

show(plotDuel)

In [7]:
f2 = Figure(title="Destination Events", plot_width=800, plot_height=600)
f2.grid.visible = False

idx = ~pd.isnull(df.EventDestination_x)

#heatmap, xedges, yedges = np.histogram2d(df.EventOrigin_x[idx], df.EventOrigin_y[idx], bins=50)
#extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

r, bins = f2.hexbin(df.EventDestination_x[idx], df.EventDestination_y[idx], size=2)

f2.add_tools(HoverTool(
    tooltips=[("count", "@c"), ("(q,r)", "(@q, @r)")],
    mode="mouse", point_policy="follow_mouse", renderers=[r]
))

show(f2)

In [106]:
df[(df.OriginPlayerID == 'Opponent13_D1') & (df.DestinationPlayerID == 'Opponent16_D5')]

,MatchID,TeamID,OriginPlayerID,DestinationPlayerID,MatchPeriod,EventTime,EventType,EventSubType,EventOrigin_x,EventOrigin_y,EventDestination_x,EventDestination_y
51284,33,Opponent16,Opponent13_D1,Opponent16_D5,2H,1699.870116,Pass,Head pass,47.0,49.0,69.0,93.0


In [95]:
for a in gDuel.nodes: print(a, '\t', len(gDuel.adj[a]))

Huskies_D1 	 17
Huskies_D10 	 4
Huskies_D2 	 16
Huskies_D3 	 14
Huskies_D4 	 11
Huskies_D5 	 13
Huskies_D6 	 12
Huskies_D7 	 16
Huskies_D8 	 8
Huskies_D9 	 5
Huskies_F1 	 16
Huskies_F2 	 13
Huskies_F3 	 4
Huskies_F4 	 10
Huskies_F5 	 13
Huskies_F6 	 9
Huskies_G1 	 4
Huskies_M1 	 19
Huskies_M10 	 4
Huskies_M11 	 4
Huskies_M12 	 11
Huskies_M13 	 4
Huskies_M2 	 3
Huskies_M3 	 20
Huskies_M4 	 16
Huskies_M5 	 0
Huskies_M6 	 14
Huskies_M7 	 0
Huskies_M8 	 11
Huskies_M9 	 11


In [100]:
infights = pd.DataFrame(index=list(gDuel.nodes))
infights['giving'] = 0
infights['receiving'] = 0

In [103]:
for index, row in df.iterrows():
    if (row.EventType == "Duel" and df.EventType[index+1] == "Duel"):
        dueler = df.OriginPlayerID[index + 1]
        if ('Husk' in row.OriginPlayerID) and ('Husk' in dueler):
            infights['giving'][row.OriginPlayerID] += 1
            infights['receiving'][dueler] += 1

In [104]:
infights

,giving,receiving
Huskies_D1,32,20
Huskies_D10,4,3
Huskies_D2,23,15
Huskies_D3,21,15
Huskies_D4,8,12
Huskies_D5,24,17
Huskies_D6,17,17
Huskies_D7,23,20
Huskies_D8,10,12
Huskies_D9,4,2


In [107]:
np.sum(infights['receiving']) / 38

12.605263157894736

In [121]:
saves = pd.DataFrame(index=[p for p in players if 'Husk' in p])
saves['count'] = 0

for (index, row) in df[(df.EventType == 'Duel') & (df.OriginPlayerID.str.contains('Huskies'))].iterrows():
    if df.EventType[index + 1] == 'Duel' and row.OriginPlayerID == df.OriginPlayerID[index + 1] and df.EventType[index + 1] == 'Duel':
        saves['count'][row.OriginPlayerID] += 1

In [122]:
saves

,count
Huskies_D1,9
Huskies_D10,2
Huskies_D2,6
Huskies_D3,3
Huskies_D4,4
Huskies_D5,3
Huskies_D6,6
Huskies_D7,7
Huskies_D8,6
Huskies_D9,1
